# GAN 직접 만들어보기!(Feat. 쉽게 씌여진 GAN)

* This Notebook referenced <code><a href='https://dreamgonfly.github.io/2018/03/17/gan-explained.html'>쉽게 씌여진 GAN</a></code>

---

* 오늘 풀어볼 문제(GAN Tutorial)
    * 0부터 9까지 숫자 모양의 손글씨 이미지를 생성하는 문제
    * Dataset: MNIST(손글씨 데이터셋으로 딥러닝계의 "Hello World!"라고 합니다!)
    * 비교적 간단한 문제지만 GAN의 원리는 어떤 문제에도 동일하게 적용되기 때문에 유용한 예제가 될 것이라고 합니다!